In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score,accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt


In [2]:
n=pd.read_csv("lending_club_loan_two.csv")
print(n)

        loan_amnt        term  int_rate  installment grade sub_grade  \
0         10000.0   36 months     11.44       329.48     B        B4   
1          8000.0   36 months     11.99       265.68     B        B5   
2         15600.0   36 months     10.49       506.97     B        B3   
3          7200.0   36 months      6.49       220.65     A        A2   
4         24375.0   60 months     17.27       609.33     C        C5   
...           ...         ...       ...          ...   ...       ...   
396025    10000.0   60 months     10.99       217.38     B        B4   
396026    21000.0   36 months     12.29       700.42     C        C1   
396027     5000.0   36 months      9.99       161.32     B        B1   
396028    21000.0   60 months     15.31       503.02     C        C2   
396029     2000.0   36 months     13.61        67.98     C        C2   

                       emp_title emp_length home_ownership  annual_inc  ...  \
0                      Marketing  10+ years           RE

In [3]:
n['loan_status'].unique()

array(['Fully Paid', 'Charged Off'], dtype=object)

In [4]:
X=n.drop(['loan_status'],axis=1)
y=n['loan_status']

In [5]:
n.isnull().sum()

,0
loan_amnt,0
term,0
int_rate,0
installment,0
grade,0
sub_grade,0
emp_title,22927
emp_length,18301
home_ownership,0
annual_inc,0


In [6]:
df1=n.dropna()
df1.isnull().sum()

,0
loan_amnt,0
term,0
int_rate,0
installment,0
grade,0
sub_grade,0
emp_title,0
emp_length,0
home_ownership,0
annual_inc,0


In [7]:
df2=df1.drop(['term','title','sub_grade','verification_status','initial_list_status','revol_util','open_acc','pub_rec','mort_acc','address','issue_d','earliest_cr_line','pub_rec_bankruptcies'],axis=1)

In [8]:
df3=df2.drop(['emp_length'],axis=1)

In [9]:
print(df3)

        loan_amnt  int_rate  installment grade                emp_title  \
0         10000.0     11.44       329.48     B                Marketing   
1          8000.0     11.99       265.68     B          Credit analyst    
2         15600.0     10.49       506.97     B             Statistician   
3          7200.0      6.49       220.65     A          Client Advocate   
4         24375.0     17.27       609.33     C  Destiny Management Inc.   
...           ...       ...          ...   ...                      ...   
396024     6000.0     13.11       202.49     B  Michael's Arts & Crafts   
396025    10000.0     10.99       217.38     B         licensed bankere   
396026    21000.0     12.29       700.42     C                    Agent   
396027     5000.0      9.99       161.32     B             City Carrier   
396028    21000.0     15.31       503.02     C     Gracon Services, Inc   

       home_ownership  annual_inc  loan_status             purpose    dti  \
0                RENT 

In [10]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

le=LabelEncoder()
df3['emp_title']=le.fit_transform(df3['emp_title'])
df3['home_ownership']=le.fit_transform(df3['home_ownership'])
df3['grade']=le.fit_transform(df3['grade'])
df3['loan_status']=le.fit_transform(df3['loan_status'])
df3['purpose']=le.fit_transform(df3['purpose'])
df3['application_type']=le.fit_transform(df3['application_type'])

In [11]:
X=df3.drop(['loan_status'],axis=1)
y=df3['loan_status']

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.8,random_state=42)

In [13]:
print(X_train)

        loan_amnt  int_rate  installment  grade  emp_title  home_ownership  \
206479    19400.0     11.14       636.42      1     113825               5   
183040    29300.0     13.98      1001.12      2      80214               1   
177719    14400.0     19.52       377.68      4     114408               5   
360317     4800.0     19.05       176.08      3      63014               5   
204438    32000.0      8.39      1008.54      0      87955               1   
...           ...       ...          ...    ...        ...             ...   
141325    13000.0     12.49       434.84      1       8478               1   
305750    12000.0     11.99       398.52      1     146439               5   
155548     6000.0     14.33       206.03      2     112082               1   
173206    11675.0     18.24       423.49      3     130788               1   
143796     7000.0     15.31       243.73      2      75577               5   

        annual_inc  purpose    dti  revol_bal  total_acc  appli

In [15]:
scaler=StandardScaler()
scaler_X_train=scaler.fit_transform(X_train)
scaler_X_test=scaler.transform(X_test)

In [16]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(scaler_X_train, y_train)

In [17]:
CV=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

In [18]:
model = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=2)

In [ ]:
model1=RandomForestClassifier()

param_grid={
    'n_estimators':[100,200,300],
    'max_depth':[5,10,15],
    'min_samples_split':[2,5,10]
}

grid_search=GridSearchCV(model1,param_grid,cv=CV,scoring='accuracy')

print(f'The model {model1} is running...')

grid_search.fit(X_resampled,y_resampled)

print(f'The {model1} is predicting...')

y_pred=grid_search.predict(scaler_X_test)

best_model=grid_search.best_estimator_

acc=accuracy_score(y_test,y_pred)
classification=classification_report(y_test,y_pred)
cm=confusion_matrix(y_test,y_pred)
print(classification)


The model RandomForestClassifier() is running...


In [ ]:
print(best_model)
print(acc)
print(cm)

In [ ]:
model.fit(scaler_X_train,y_train)

In [ ]:
y_pred=model.predict(scaler_X_test)

In [ ]:
acc=accuracy_score(y_test,y_pred)
classification=classification_report(y_test,y_pred)
cm=confusion_matrix(y_test,y_pred)
print(classification)
roc_auc=roc_auc_score(y_test,y_pred)

In [ ]:
import pickle

pickle.dump(best_model,open('model.pkl','wb'))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = best_model.predict(scaler_X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))